In [1]:
import pandas as pd
import pandas_gbq

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from Variables.create_vars import *
from BigQuery_Classes.bq_client import BQ_Client_Connect

In [2]:
connect_client = BQ_Client_Connect()
client = connect_client.create_client()

tables = client.list_tables(bq_dataset_id)

In [3]:
cpi_table_names = []
for table in tables:
    cpi_table_names.append("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))  

In [4]:
cpi_table_names = [i for i in cpi_table_names if not ('current_0' in i or 'summaries_2' in i)]  

In [5]:
cpi_table_names

['cpi-tfx.BLS_Data.all_items_1',
 'cpi-tfx.BLS_Data.apparel_13',
 'cpi-tfx.BLS_Data.commodities_services_special_20',
 'cpi-tfx.BLS_Data.education_and_communication_17',
 'cpi-tfx.BLS_Data.food_beverage_11',
 'cpi-tfx.BLS_Data.housing_12',
 'cpi-tfx.BLS_Data.medical_15',
 'cpi-tfx.BLS_Data.other_goods_and_services_18',
 'cpi-tfx.BLS_Data.recreation_16',
 'cpi-tfx.BLS_Data.transportation_14']

In [6]:
# plot_heights= [500, 2000, 2500]
plot_heights= [500, 2000, 2500, 2000, 12000, 4000, 2000, 2000, 4000, 2500]
height_counter = 0

In [7]:
counter = 0
i = 1
j = 1
k = 1

In [10]:
for cpi_table in cpi_table_names:
    query = '''SELECT * 
            FROM `{}`
            '''.format(cpi_table)
    data = pd.read_gbq(query = query, project_id = 'cpi-tfx')
    data.sort_values(by = ['date'], inplace = True)
    data.reset_index(drop = True, inplace = True)

    data_item_list = data['item'].unique()

    if len(data_item_list)%2 == 0:
        columns = 2
        rows = int(len(data_item_list)/2)

    else:
        columns = 2
        rows = int((len(data_item_list) + 1 )/2)

    fig = make_subplots(rows = rows, 
                        cols = columns,
                        subplot_titles = data_item_list)

    counter = 0
    i = 1
    j = 1
    k = 1

    while counter <= len(data_item_list):

        if k <= columns:

            fig.add_trace(go.Scatter(x = data[data['item'] == '{}'.format(data_item_list[counter])]['date'], 
                                    y = data[data['item'] == '{}'.format(data_item_list[counter])]['value'],
                                    mode = 'lines+markers'),
                                    row = i, 
                                    col = j)

            fig.update_xaxes(title_text = 'Date', row = i, col = j)
            fig.update_yaxes(title_text = 'Index', row = i, col = j)

            counter = counter + 1
            j = j + 1
            k = k + 1


        elif k > columns:
            k = 1
            counter = counter + 1
            i = i + 1
            j = 1
        
            fig.add_trace(go.Scatter(x = data[data['item'] == '{}'.format(data_item_list[counter])]['date'], 
                                    y = data[data['item'] == '{}'.format(data_item_list[counter])]['value'],
                                    mode = 'lines+markers'),
                                    row = i, 
                                    col = j)

            fig.update_xaxes(title_text = 'Date', row = i, col = j)
            fig.update_yaxes(title_text = 'Index', row = i, col = j)

            k = k + 1
            j = j + 1

    fig.update_layout(height = plot_heights[height_counter], 
                      showlegend = False,
                      title={'text': '<b>{}</b>'.format(cpi_table),
                             'x':0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'})

    height_counter = height_counter + 1

    fig.show()

IndexError: index 3 is out of bounds for axis 0 with size 2